In [1]:
import pandas as pd


In [2]:
sheet_data = pd.read_excel(r"C:\Users\fsp_adolpho.salvador\Desktop\Konica Minolta\Desktop Cloud - Documentos\Desktop\Py\Pendencias\Arquivos\Mapa2.xlsx", sheet_name='Sheet1')

sheet_data.columns

Index([           'Unnamed: 0',              'Material',
                  'CE03_zva70',            'CE04_zva70',
                  'CE05_zva70',            'CE10_zva70',
                  'CE11_zva70',            'CE13_zva70',
                 'Denominação',            'Criado por',
                       'Canal',     'Pendências Gerais',
                  'CE01_zstok',            'CE03_zstok',
                  'CE05_zstok',            'CE07_zstok',
                  'CE09_zstok',            'CE10_zstok',
                  'CE11_zstok',            'CE13_zstok',
               'Contr.qualid.',     2025-04-22 00:00:00,
           2025-05-05 00:00:00,     2025-05-06 00:00:00,
           2025-05-07 00:00:00,     2025-05-12 00:00:00,
           2025-06-02 00:00:00,     2025-06-04 00:00:00,
           2025-06-05 00:00:00,     2025-06-16 00:00:00,
           2025-06-17 00:00:00,     2025-06-18 00:00:00,
           2025-06-20 00:00:00,     2025-06-22 00:00:00,
           2025-06-23 00:00:00,

In [3]:
sheet_data['pendencia_total'] = sheet_data.filter(like='_zva70').sum(axis=1)

sheet_data['estoque_total'] = sheet_data.filter(like='_zstok').sum(axis=1)

sheet_data['FUP_total'] = sheet_data.filter(like='_FUP').sum(axis=1)

sheet_data['estoque_no_similar'] = sheet_data.apply(
    lambda row: row['Quant estoque'] - row['estoque_total'] if row['Quant estoque'] > 0 else 0,
    axis=1
)


In [4]:
sheet_data

,Unnamed: 0,Material,CE03_zva70,CE04_zva70,CE05_zva70,CE10_zva70,CE11_zva70,CE13_zva70,Denominação,Criado por,...,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Quant estoque,pendencia_total,estoque_total,FUP_total,estoque_no_similar
0,0,08AA77360,4,0,0,0,0,0,ENGRENAGEM,BRENDA DANTAS SOUZA,...,0,0,0,0,0,0,4,12,0.0,0
1,1,20AL48470G,2,0,0,0,0,0,BOTÃO,BRENDA DANTAS SOUZA,...,-,-,-,0,1,89,2,17,0.0,72
2,2,25AAR70300,0,0,0,0,3,0,FIO,JESSIE OLIVEIRA SILVA,...,-,-,-,0,1,0,3,0,0.0,0
3,3,26NA32870,0,0,1,0,0,0,MOLA DE PRESSÃO 2,BRENDA DANTAS SOUZA,...,-,-,-,0,1,0,1,22,0.0,0
4,4,4030300501,0,0,0,0,3,0,ROLO,ROBERTA MACHADO,...,0,0,0,0,0,0,3,17,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,139,LY5606001,0,0,0,0,1,0,UNID FIXAÇÃO MOD HL5470DW/5472/8712/6182,MARIANA NUNES DA SILVA,...,0,0,0,0,0,0,1,0,0.0,0
140,140,V500010073,0,0,0,0,14,0,PRESILHA,REGILENE DE CASSIA LUIZ OLIVEIRA/REGILENE DE C...,...,0,0,0,0,0,0,14,0,0.0,0
141,141,V500010081,0,3,0,0,0,0,PRESILHA,JULIA MARIA MORETO DA SILVA,...,0,0,0,0,0,0,3,0,0.0,0
142,142,V500020030,0,0,0,0,14,0,GARRA,REGILENE DE CASSIA LUIZ OLIVEIRA/REGILENE DE C...,...,0,0,0,0,0,0,14,0,0.0,0


In [5]:
sheet_data['projecao_compra'] = sheet_data['pendencia_total'] - (sheet_data['estoque_total'] + sheet_data['FUP_total'] + sheet_data['estoque_no_similar'])
sheet_data['projecao_compra'] = sheet_data['projecao_compra'].apply(lambda x: max(x, 0))

In [6]:
sheet_data

,Unnamed: 0,Material,CE03_zva70,CE04_zva70,CE05_zva70,CE10_zva70,CE11_zva70,CE13_zva70,Denominação,Criado por,...,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Quant estoque,pendencia_total,estoque_total,FUP_total,estoque_no_similar,projecao_compra
0,0,08AA77360,4,0,0,0,0,0,ENGRENAGEM,BRENDA DANTAS SOUZA,...,0,0,0,0,0,4,12,0.0,0,0.0
1,1,20AL48470G,2,0,0,0,0,0,BOTÃO,BRENDA DANTAS SOUZA,...,-,-,0,1,89,2,17,0.0,72,0.0
2,2,25AAR70300,0,0,0,0,3,0,FIO,JESSIE OLIVEIRA SILVA,...,-,-,0,1,0,3,0,0.0,0,3.0
3,3,26NA32870,0,0,1,0,0,0,MOLA DE PRESSÃO 2,BRENDA DANTAS SOUZA,...,-,-,0,1,0,1,22,0.0,0,0.0
4,4,4030300501,0,0,0,0,3,0,ROLO,ROBERTA MACHADO,...,0,0,0,0,0,3,17,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,139,LY5606001,0,0,0,0,1,0,UNID FIXAÇÃO MOD HL5470DW/5472/8712/6182,MARIANA NUNES DA SILVA,...,0,0,0,0,0,1,0,0.0,0,1.0
140,140,V500010073,0,0,0,0,14,0,PRESILHA,REGILENE DE CASSIA LUIZ OLIVEIRA/REGILENE DE C...,...,0,0,0,0,0,14,0,0.0,0,14.0
141,141,V500010081,0,3,0,0,0,0,PRESILHA,JULIA MARIA MORETO DA SILVA,...,0,0,0,0,0,3,0,0.0,0,3.0
142,142,V500020030,0,0,0,0,14,0,GARRA,REGILENE DE CASSIA LUIZ OLIVEIRA/REGILENE DE C...,...,0,0,0,0,0,14,0,0.0,0,14.0


In [7]:
valores_unicos = sheet_data['estoque_no_similar'].unique()
valores_unicos

array([    0,    72,    93,    59,     8,    12, 13369,  1149,    16,
         569,   344,    95,  4190,  1871,     6,   118], dtype=int64)

In [8]:
caminho_saida = r'C:\Users\fsp_adolpho.salvador\Desktop\Konica Minolta\Desktop Cloud - Documentos\Desktop\Py\Analise\Pendencia\Mapa Analisado1.0.xlsx'
sheet_data.to_excel(caminho_saida, index=True)

In [9]:
def atender_estoque_ajustado_v2(row):
    if row['projecao_compra'] == 0:
        for col in ['CE05_zstok', 'CE13_zstok', 'CE11_zstok', 'CE09_zstok', 'CE07_zstok', 'CE01_zstok', 'estoque_total', 'estoque_no_similar']:
            if row[col] >= row['pendencia_total']:
                if col == 'estoque_no_similar':
                    return "Atender Pelo Similar"
                else:
                    return f"Atender pelo {col.split('_')[0]}"
        if row['FUP_total'] >= row['pendencia_total']:
            return "Aguardar IMP"
        else:
            return "Sem estoque disponível"
    else:
        return row['projecao_compra']  

sheet_data['priorizacao_estoque'] = sheet_data.apply(atender_estoque_ajustado_v2, axis=1)



In [10]:
sheet_data

,Unnamed: 0,Material,CE03_zva70,CE04_zva70,CE05_zva70,CE10_zva70,CE11_zva70,CE13_zva70,Denominação,Criado por,...,Unnamed: 15,Unnamed: 16,Unnamed: 17,Quant estoque,pendencia_total,estoque_total,FUP_total,estoque_no_similar,projecao_compra,priorizacao_estoque
0,0,08AA77360,4,0,0,0,0,0,ENGRENAGEM,BRENDA DANTAS SOUZA,...,0,0,0,0,4,12,0.0,0,0.0,Atender pelo CE05
1,1,20AL48470G,2,0,0,0,0,0,BOTÃO,BRENDA DANTAS SOUZA,...,-,0,1,89,2,17,0.0,72,0.0,Atender pelo CE11
2,2,25AAR70300,0,0,0,0,3,0,FIO,JESSIE OLIVEIRA SILVA,...,-,0,1,0,3,0,0.0,0,3.0,3.0
3,3,26NA32870,0,0,1,0,0,0,MOLA DE PRESSÃO 2,BRENDA DANTAS SOUZA,...,-,0,1,0,1,22,0.0,0,0.0,Atender pelo CE05
4,4,4030300501,0,0,0,0,3,0,ROLO,ROBERTA MACHADO,...,0,0,0,0,3,17,0.0,0,0.0,Atender pelo CE13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,139,LY5606001,0,0,0,0,1,0,UNID FIXAÇÃO MOD HL5470DW/5472/8712/6182,MARIANA NUNES DA SILVA,...,0,0,0,0,1,0,0.0,0,1.0,1.0
140,140,V500010073,0,0,0,0,14,0,PRESILHA,REGILENE DE CASSIA LUIZ OLIVEIRA/REGILENE DE C...,...,0,0,0,0,14,0,0.0,0,14.0,14.0
141,141,V500010081,0,3,0,0,0,0,PRESILHA,JULIA MARIA MORETO DA SILVA,...,0,0,0,0,3,0,0.0,0,3.0,3.0
142,142,V500020030,0,0,0,0,14,0,GARRA,REGILENE DE CASSIA LUIZ OLIVEIRA/REGILENE DE C...,...,0,0,0,0,14,0,0.0,0,14.0,14.0


In [11]:
caminho_saida = r'C:\Users\fsp_adolpho.salvador\Desktop\Konica Minolta\Desktop Cloud - Documentos\Desktop\Py\Analise\Pendencia\Arquivos\Mapa Analisado2.0.xlsx'

sheet_data.to_excel(caminho_saida, index=True)